In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import scipy
from scipy.stats import chisquare
import sys

def PSR_NGR(positions, addonfactor, singleposition):

    GrossExposure = 0
    CurrentExposure = 0
    NetExposure = 0
    AddOn_Gross = 0

    if singleposition == True:
        GrossExposure += max(0, positions[0])
        CurrentExposure += positions[0]
        AddOn_Gross += addonfactor * positions[1]
    else:
        for i in range(len(positions)):
            GrossExposure += max(0, positions[i,0])
            CurrentExposure += positions[i,0]
            AddOn_Gross += addonfactor * positions[i,1]

    NetExposure = max(0, CurrentExposure)

    if GrossExposure == 0:
        NGR = 0
    else:
        NGR = NetExposure/GrossExposure
    
    PSR = max(0, CurrentExposure) + NGR * (0.6) * (AddOn_Gross) + (0.4) * (AddOn_Gross)

    return PSR

def PSR_Normal(positions, addonfactor, singleposition):

    MtM = 0
    AddOn = 0

    if singleposition == True:
        MtM += positions[0]
        AddOn += addonfactor * positions[1]
    else:
        for i in range(len(positions)):
            MtM += positions[i,0]
            AddOn += addonfactor * positions[i,1]

    CurrentExposure = max(0,MtM)

    PSR = CurrentExposure + AddOn
    
    return PSR


def PSR_Conservative(positions, addonfactor):

    total_PSR_NGR = 0
    total_PSR_Normal = 0
    singleposition = True

    for i in range(len(positions)):
        total_PSR_NGR += PSR_NGR(positions[i,:], addonfactor, singleposition)
        total_PSR_Normal += PSR_Normal(positions[i,:], addonfactor, singleposition)
    
    return [total_PSR_NGR, total_PSR_Normal]

def PSR_Linear(positions, addonfactor):

    total_PSR_NGR = 0
    total_PSR_Normal = 0

    base_PSR_NGR = 0
    base_PSR_Normal = 0

    singleposition = True
    sample = positions[0,:]
    base_PSR_NGR += PSR_NGR(sample, addonfactor, singleposition)
    total_PSR_NGR = base_PSR_NGR

    base_PSR_Normal += PSR_Normal(sample, addonfactor, singleposition)
    total_PSR_Normal = base_PSR_Normal
    singleposition = False

    for i in range(len(positions)-1):
        total_PSR_NGR += (PSR_NGR(positions[[0,i+1], :], addonfactor, singleposition) - base_PSR_NGR)
        total_PSR_Normal += (PSR_Normal(positions[[0,i+1], :], addonfactor, singleposition) - base_PSR_Normal)


    return [total_PSR_NGR, total_PSR_Normal]


def PSR_Average(positions, addonfactor, n):

    total_PSR_NGR = 0
    total_PSR_Normal = 0
    base_PSR_NGR = 0
    base_PSR_Normal = 0

    singleposition = True
    base_PSR_NGR += PSR_NGR(positions[0,:], addonfactor, singleposition)
    total_PSR_NGR = base_PSR_NGR

    base_PSR_Normal += PSR_Normal(positions[0,:], addonfactor, singleposition)
    total_PSR_Normal = base_PSR_Normal
    singleposition = False

    for i in range(len(positions)-1):

        # take out the baseline position and the x_i position
        trade_i = positions[[0,i+1], :]
        
        # multiply the i-th position by n - as it happens in the formula
        trade_i[1,:] *= n

        # multiply the formula by n
        total_PSR_NGR += 1/n*(PSR_NGR(trade_i, addonfactor, singleposition) - base_PSR_NGR)
        total_PSR_Normal += 1/n*(PSR_Normal(trade_i, addonfactor, singleposition) - base_PSR_Normal)


    return [total_PSR_NGR, total_PSR_Normal]



#### Define the main program below

In [2]:
def main(n, baseline_mtm):


    # TEST 1: PSR_NGR = 30.765 , PSR_BruteForce = 30.9
    '''    
    positions = np.zeros((4,2))

    positions[0,0] = 10
    positions[1,0] = 20
    positions[2,0] = -10
    positions[3,0] = 10

    
    positions[0,1] = 20
    positions[1,1] = 40
    positions[2,1] = 10
    positions[3,1] = 20
    '''
    
    # Randomly generated set of [MTM, Notional]
   
    positions = np.zeros((n,2))

    positions[0,0] = baseline_mtm
    positions[0,1] = 1000

    for i in range(len(positions)-1):
        positions[i+1,0] = random.randint(-10,10)
        positions[i+1,1] = abs(5*positions[i,0])


    
    # TEST 1A
    '''
    positions = np.zeros((2,2))

    positions[0,0] = 10
    positions[1,0] = 20
    
    positions[0,1] = 20
    positions[1,1] = 40
    '''

    # TEST 2: PSR_NGR = 15.51 , PSR_BruteForce = 15.6

    '''
    positions = np.zeros((3,2))

    positions[0,0] = 10
    positions[1,0] = -5
    positions[2,0] = 10
    
    positions[0,1] = 20
    positions[1,1] = 10
    positions[2,1] = 30


    '''
    addonfactor = 0.01
    singleposition = False

    NGR_BF = PSR_NGR(positions, addonfactor, singleposition)
    Normal_BF = PSR_Normal(positions, addonfactor, singleposition)

    [NGR_Lin, Normal_Lin] = PSR_Linear(positions, addonfactor)
    [NGR_Cons, Normal_Cons] = PSR_Conservative(positions, addonfactor)
    [NGR_Avg, Normal_Avg] = PSR_Average(positions, addonfactor, n)



    return [positions, addonfactor, NGR_BF, NGR_Lin, NGR_Cons, NGR_Avg, Normal_BF, Normal_Lin, Normal_Cons, Normal_Avg]


[positions, addonfactor, NGR_BF, NGR_Lin, NGR_Cons, NGR_Avg, Normal_BF, Normal_Lin, Normal_Cons, Normal_Avg] = main(20, 100)

print("\n")
print("[PSR NGR, PSR Normal] for Brute Force: ", [NGR_BF, Normal_BF])
print("\n")

print("[PSR NGR, PSR Normal] for Linearisation: ", PSR_Linear(positions, addonfactor))
print("\n")
print("[PSR NGR, PSR Normal] for Conservative: ", PSR_Conservative(positions, addonfactor))
print("\n")
print("[PSR NGR, PSR Normal] for Averages: ", PSR_Average(positions, addonfactor, 20))

print("\n")
print("Positions Matrix: \n", positions)




[PSR NGR, PSR Normal] for Brute Force:  [162.9642372881356, 165.15]


[PSR NGR, PSR Normal] for Linearisation:  [162.96930000000003, 165.15000000000003]


[PSR NGR, PSR Normal] for Conservative:  [193.01000000000002, 197.14999999999998]


[PSR NGR, PSR Normal] for Averages:  [168.98999999999998, 174.15]


Positions Matrix: 
 [[ 100. 1000.]
 [  -7.  500.]
 [   9.   35.]
 [  -6.   45.]
 [   0.   30.]
 [   4.    0.]
 [ -10.   20.]
 [  10.   50.]
 [  10.   50.]
 [  -6.   50.]
 [   4.   30.]
 [   0.   20.]
 [   5.    0.]
 [  -3.   25.]
 [   2.   15.]
 [   8.   10.]
 [  10.   40.]
 [   5.   50.]
 [   4.   25.]
 [   6.   20.]]


#### Perform simluations of the code and store the results to get an idea of how much the difference NGR makes in the long run

In [3]:
def graph(simulations, baseline_mtm, lin=True, cons=True, bf = True, avg=True):


    diff_bf = []
    diff_lin = []
    diff_cons = []
    diff_avg = []

    for i in range(simulations - 5):

        [positions, addonfactor, NGR_BF, NGR_Lin, NGR_Cons, NGR_Avg, Normal_BF, Normal_Lin, Normal_Cons, Normal_Avg] = main(i+5, baseline_mtm)

        diff_bf.append(Normal_BF - NGR_BF)
        diff_lin.append(Normal_Lin - NGR_Lin)
        diff_cons.append(Normal_Cons - NGR_Cons)
        diff_avg.append(Normal_Avg - NGR_Avg)


    x = np.linspace(0, simulations - 5, simulations - 5)
    plt.figure(figsize=(10,8))
    plt.grid(True)

    if bf == True:
        plt.plot(x,diff_bf, color="b", label="Brute Force")

    if lin == True:
        plt.plot(x,diff_lin, color="g", label="Linearisation")
    
    if cons == True:
        plt.plot(x,diff_cons, color="r", label="Conservative")
    
    if avg == True:
        plt.plot(x,diff_avg, color="y", label="Averages")

    plt.xlabel("Number of trades")
    plt.title("Difference between Normal PSR and NGR")
    plt.ylabel("Difference")
    plt.legend()
    plt.show()
    

In [4]:
widgets.interact(graph, 
         simulations = widgets.IntSlider(min=5,max=250,step=1,value=100,description="Simulations"),
         baseline_mtm = widgets.IntSlider(min=10,max=1000,step=10,value=100,description="Base MtM"),)

interactive(children=(IntSlider(value=100, description='Simulations', max=250, min=5), IntSlider(value=100, de…

<function __main__.graph(simulations, baseline_mtm, lin=True, cons=True, bf=True, avg=True)>

If the baseline MTM is around 100, then we get a lovely graph showing the different methods all grow at linear rates. 

However, if I set the baseline mtm equal to 1000 or 10, the graphs are incredibly bizarre. Why is this occuring??